## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 TensorFlow 建立鳶尾花（Iris）分類模型並將其導出為 ONNX 格式的範例。該模型使用簡單的全連接層來進行分類，並轉換為 ONNX 格式，方便在 ONNX-MLIR 或其他 ONNX 支持的推理引擎上運行。

### 1.1 安裝必要的套件
如果尚未安裝 tensorflow 和 tf2onnx，可以使用以下命令安裝：



In [ ]:
!pip install tensorflow tf2onnx

### 1.2 建立並訓練 TensorFlow 模型
以下程式碼將建立一個簡單的神經網絡來分類鳶尾花數據集，並將其導出為 ONNX 格式。

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 載入鳶尾花資料集
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),  # 4 個特徵
    tf.keras.layers.Dense(10, activation='relu'),  # 隱藏層
    tf.keras.layers.Dense(3, activation='softmax') # 輸出層，3 個分類
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# 評估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"模型準確率: {accuracy:.2f}")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108d0f3d0>>
Traceback (most recent call last):
  File "/Users/yilintsai/anaconda3/envs/onnx-mlir/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


### 1.3 將模型轉換為 ONNX 格式
使用 tf2onnx 將訓練好的 TensorFlow 模型轉換為 ONNX 格式：

In [2]:
import tf2onnx

# 將 Keras 模型轉換為 ONNX 格式
spec = (tf.TensorSpec((None, 4), tf.float32, name="float_input"),)  # 定義輸入規範
output_path = "tf_model.onnx"  # 輸出 ONNX 模型的路徑

# 轉換模型
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"ONNX 模型已保存至 {output_path}")

ONNX 模型已保存至 tf_model.onnx


2024-11-11 21:26:54.075651: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-11 21:26:54.075807: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-11 21:26:54.097347: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-11 21:26:54.097453: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


### 1.4 使用ONNX Runtime進行推論測試
我們可以先透過 ONNX Runtime 輸入一筆測試資料檢查推論結果。可以跟稍後 ONNX-MLIR 推論結果進行驗證比較看有沒有數值一至。

In [2]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('tf_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})

# 輸出預測結果
print(pred_onnx)

[array([[0.00611059, 0.27653578, 0.71735364]], dtype=float32)]


## 2. 使用 ONNX-MLIR 轉換模型為共享庫
以下是如何使用 ONNX-MLIR 將 tf.onnx 模型轉換為共享庫（.so 文件）的步驟。

### 2.1 將 ONNX 模型編譯為共享庫
使用 onnx-mlir 將 tf_model.onnx 模型轉換為共享庫（.so 文件）。
> 成功輸出後即可前往第3撰寫 C++ 程式進行推論

執行以下命令：

In [112]:
!../onnx-mlir/Release/bin/onnx-mlir --EmitLib tf_model.onnx

[1/6] Sat Nov 16 21:49:33 2024 (0s) Importing ONNX Model to MLIR Module from "tf_model.onnx"
[2/6] Sat Nov 16 21:49:33 2024 (0s) Compiling and Optimizing MLIR Module
[3/6] Sat Nov 16 21:49:33 2024 (0s) Translating MLIR Module to LLVM and Generating LLVM Optimized Bitcode
[4/6] Sat Nov 16 21:49:34 2024 (1s) Generating Object from LLVM Bitcode
[5/6] Sat Nov 16 21:49:34 2024 (1s) Linking and Generating the Output Shared Library
[6/6] Sat Nov 16 21:49:34 2024 (1s) Compilation completed


檢查生成的 tf_model.so 是否依賴動態庫。在上述編譯過程中，由於 onnx-mlir 默認以靜態方式將 libcruntime.a 包含進去，因此生成的 tf_model.so 並不顯式依賴 libcruntime.so。從以下輸出可以看出，它僅依賴系統動態庫，例如 libc++ 和 libSystem.B.dylib：

```
tf_model.so:
	tf_model.so (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libc++.1.dylib (compatibility version 1.0.0, current version 1500.65.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1319.100.3)
```

如果在 onnx-mlir 生成 tf_model.so 的過程中未將 libcruntime.a 靜態鏈接，而是以動態方式依賴外部庫，輸出的結果應包含對 libcruntime.dylib 的依賴，例如：

```
tf_model.so:
	tf_model.so (compatibility version 0.0.0, current version 0.0.0)
	/path/to/libcruntime.dylib (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libc++.1.dylib (compatibility version 1.0.0, current version 1500.65.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1319.100.3)

```

In [114]:
# mac 使用 otool 
# Linux 使用 ldd
!otool -L tf_model.so

tf_model.so:
	tf_model.so (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libc++.1.dylib (compatibility version 1.0.0, current version 1500.65.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1319.100.3)


In [125]:
!ld tf_model.so

ld: warning: platform not specified
ld: warning: -arch not specified
ld: warning: No platform min-version specified on command line
ld: warning: ignoring file tf_model.so, building for -unknown but attempting to link with file built for unknown-unsupported file format ( 0x7F 0x45 0x4C 0x46 0x02 0x01 0x01 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 )
Undefined symbols for architecture unknown:
  "_main", referenced from:
     implicit entry/start for main executable
ld: symbol(s) not found for architecture unknown


In [116]:
# 使用strip工具移除不必要的符號
!strip -xS tf_model.so

In [118]:
!size tf_model.so

__TEXT	__DATA	__OBJC	others	dec	hex
16384	16384	0	21312	54080	d340


### 2.2 ONNX 模型編譯為物件檔案
將 ONNX 模型編譯成物件檔案（.o 檔案）。其中，--EmitObj 選項指定輸出為物件檔案。這樣，可以將產生的物件檔案與其他程式碼進行連結，形成可執行檔案或共享庫。

In [119]:
!../onnx-mlir/Release/bin/onnx-mlir --EmitObj tf_model.onnx

[1/5] Sat Nov 16 21:50:13 2024 (0s) Importing ONNX Model to MLIR Module from "tf_model.onnx"
[2/5] Sat Nov 16 21:50:13 2024 (0s) Compiling and Optimizing MLIR Module
[3/5] Sat Nov 16 21:50:13 2024 (0s) Translating MLIR Module to LLVM and Generating LLVM Optimized Bitcode
[4/5] Sat Nov 16 21:50:13 2024 (0s) Generating Object from LLVM Bitcode
[5/5] Sat Nov 16 21:50:13 2024 (0s) Compilation completed


產生出物件檔案後可以使用使用 C++ 編譯器將 tf_model.o 物件檔案連結成共享庫 tf_model.so。其中，`-shared` 選項指定輸出為共享庫，`-fPIC` 選項確保產生的位置獨立碼（Position Independent Code），-L../onnx-mlir/Release/lib 指定連結時搜尋的庫目錄，`-lcruntime` 則連結名為 libcruntime.a 的靜態庫。

In [120]:
# 產生出tf_model.so共享庫，等同於2.1產物
!c++ tf_model.o -o tf_model.so -shared -fPIC -L../onnx-mlir/Release/lib -lcruntime

In [ ]:
# 撰寫 C++ 程式進行推論（這部分內容等同於3.2.1結果）
!g++ --std=c++17 inference.cpp tf_model.o -o main -I ../onnx-mlir/include -L../onnx-mlir/Release/lib -lcruntime
!./main

### 2.3 編譯適用於 Linux x86_64 架構的共享庫
首先在 macOS 上使用 onnx-mlir 將 tf_model.onnx 編譯成適用於 x86_64 架構的 Linux 物件檔案。

In [129]:
!../onnx-mlir/Release/bin/onnx-mlir --EmitObj tf_model.onnx --mtriple=x86_64-linux-gnu-g++

[1/5] Sat Nov 16 21:55:53 2024 (0s) Importing ONNX Model to MLIR Module from "tf_model.onnx"
[2/5] Sat Nov 16 21:55:53 2024 (0s) Compiling and Optimizing MLIR Module
[3/5] Sat Nov 16 21:55:53 2024 (0s) Translating MLIR Module to LLVM and Generating LLVM Optimized Bitcode
[4/5] Sat Nov 16 21:55:53 2024 (0s) Generating Object from LLVM Bitcode
[5/5] Sat Nov 16 21:55:53 2024 (0s) Compilation completed


In [130]:
# 接著透過交叉編譯，產生在 Linux x86_64 平台上可被運行的共享庫（.so 文件）
!x86_64-unknown-linux-gnu-gcc tf_model.o -shared -o tf_model.so

## 3. 撰寫 C++ 程式進行推論
### 3.1 撰寫 C++ 程式

> 請參考 tf-example/inference.cpp

### 3.2 編譯程式
要編譯上述 C++ 程式碼，除了確保 onnx-mlir 生成的 .so 模型文件（例如 tf_model.so）在當前目錄，還需要指定 OnnxMlirRuntime.h 頭文件的路徑。因為程式中引用了 OnnxMlirRuntime.h，因此需要使用 -I 選項指定頭文件的路徑。如果您將 ONNX-MLIR 的安裝位置放在 onnx-mlir/include 目錄中，可以使用以下命令進行編譯：

1. **`tf_model.so` 的定位**：
   - 它是模型推論的主要邏輯庫。
   - 它依賴 **`libcruntime`**，無論是靜態鏈接還是動態鏈接。

2. **`libcruntime` 的角色**：
   - 提供運行時支持，包括 **張量創建**、**內存管理** 和 **數據操作**。
   - 必須在程式中靜態或動態鏈接。

3. **`OnnxMlirRuntime.h` 的作用**：
   - 定義了與 **`libcruntime`** 和模型共享庫（如 `tf_model.so`）交互的 API。
   - 並不包含任何實際功能，功能由 **`libcruntime`** 提供。

#### 3.2.1 靜態編譯共享庫（直接鏈接共享庫）
此方法在 C++ 程式中通過 extern "C" 引用 libcruntime 提供的函數，並直接將 tf_model.so 作為共享庫進行鏈接。由於 tf_model.so 已經靜態包含了 libcruntime 的功能，因此不需要額外鏈接 libcruntime。

In [88]:
!g++ --std=c++17 inference.cpp tf_model.so -o main -I../onnx-mlir/include
!./main # 94kb

模型輸出：0.00611059 0.276536 0.717354 


#### 3.2.2 靜態鏈接編譯（使用靜態庫）
此方法使用 dlopen 在程式執行時動態載入 tf_model.so，但所有 libcruntime 函數（如 OMTensorCreate）已經在編譯階段通過靜態鏈接嵌入可執行文件。

In [41]:
!g++ --std=c++17 static-inference.cpp -o main -I../onnx-mlir/include ../onnx-mlir/Release/lib/libcruntime.a
!./main # 115kb

模型輸出：0.00611059 0.276536 0.717354 


In [49]:
# 另一種寫法
!g++ --std=c++17 static-inference.cpp -o main -I../onnx-mlir/include -L../onnx-mlir/Release/lib -lcruntime
!./main # 115kb

Error loading shared library: dlopen(./tf_model.so, 0x0001): tried: './tf_model.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS./tf_model.so' (no such file), '/usr/lib/./tf_model.so' (no such file, not in dyld cache), './tf_model.so' (no such file), '/Users/yilintsai/Desktop/GitHub/1010code-github/onnx-mlir-tutorial/tf-example/tf_model.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/yilintsai/Desktop/GitHub/1010code-github/onnx-mlir-tutorial/tf-example/tf_model.so' (no such file), '/Users/yilintsai/Desktop/GitHub/1010code-github/onnx-mlir-tutorial/tf-example/tf_model.so' (no such file)


#### 3.2.3 動態載入共享庫（dlopen 模式）
此方法在編譯時不進行任何靜態或動態庫的鏈接，完全依賴程式運行時使用 dlopen 動態載入 tf_model.so 和 libcruntime.so。這種方式使得可執行文件的體積最小，但需要在執行時確保共享庫的可用性。

> 尚未成功編譯出 libcruntime.so 未確定官方是否支援

In [45]:
!g++ --std=c++17 dlopen-inference.cpp -o main -I../onnx-mlir/include
# !./main # 40 kb